# Activité guidée – Player + main.py (sans animations)
_Mise à jour : 2025-09-28 09:30_

**Contexte** : `Map`, `Gui` et `Entity` sont fournis. On **reportera les animations** à plus tard.

**Objectifs** :
- Implémenter `Player` (lecture JSON, position, limites carte) **sans animations**.
- Compléter `main.py` : créer le joueur et coder `on_key_press()` / `on_key_release()` pour le déplacer.
Vous pourrez **tester le déplacement** du joueur immédiatement.


## Sommaire
1. [Rappel express : Entity & déplacements](#rappel)
2. [Tâche 1 : Player (sans animations)](#player)
3. [Tâche 2 : Compléter main.py (création du joueur + touches)](#main)
4. [Checklist de test](#check)


## 1) Rappel express : Entity & déplacements <a id='rappel'></a>

- `Entity` (hérite de `arcade.Sprite`) fournit : textures (qu’on **n’utilise pas encore**), position (`center_x`, `center_y`), état `status`, indice `current_texture_indice`, et `attributes` (dict).
- `change_x` et `change_y` sont les **déplacements par frame** :
  - `change_x > 0` → droite, `< 0` → gauche ; `change_y > 0` → haut, `< 0` → bas.
  - À chaque frame, Arcade fait : `center_x += change_x` et `center_y += change_y`.


## 2) Tâche 1 : Player (sans animations) <a id='player'></a>

Créez `player.py` avec une classe `Player` **héritant** de `Entity`.
À faire :
1. Lire `PLAYER_CARACTERISTICS_FILE` (JSON) et remplir `self.attributes`.
2. Positionner le joueur avec `Init_x`/`Init_y` (tenir compte de `MAP_SCALING`).
3. Empêcher le joueur de sortir de la carte (bornes).
4. Garder simplement la **première texture** (pas d’animations pour l’instant).

👉 **Complétez les zones `# TODO`**.


In [ ]:
# === player.py (sans animations) ===
from entity import *
from constants import *
import json

class Player(Entity):
    def __init__(self, file_name, scaling, img_width, img_height, coords):
        super().__init__(file_name, scaling, img_width, img_height, coords)
        self.status = 0
        self.init_x_pos = 0
        self.init_y_pos = 0

    def setup(self):
        super().setup()
        # TODO: lire le JSON et charger les caractéristiques "Player" dans self.attributes
        f = open(PLAYER_CARACTERISTICS_FILE)
        data = json.load(f)
        for key, value in data["Player"].items():
            self.attributes[key] = value
        f.close()

        # TODO: positionner le joueur en tenant compte de l'échelle
        self.center_x = self.attributes["Init_x"] * MAP_SCALING
        self.init_x_pos = self.center_x
        ############ A COMPLETER (ORDONNEES) #############

        # Texture de départ : 1re frame (pas d'animation ici)
        self.status = 0
        self.current_texture_indice = 0
        self.texture = self.textures[self.status][self.current_texture_indice]

    def update(self):
        # Empêcher de sortir de la carte
        if self.center_x < 0:
            self.center_x = 0
        elif self.center_x > MAP_WIDTH * MAP_SCALING:
            self.center_x = MAP_WIDTH * MAP_SCALING

        ############ A COMPLETER (ORDONNEES) #############
        
        # Rien d'autre ici (animations plus tard)
        return


## 3) Tâche 2 : Compléter `main.py` (création du joueur + touches) <a id='main'></a>

Dans `main.py`, vous devez :
1. **Créer le joueur** et l’ajouter à `sprites_list`, puis appeler `setup()`.
2. Coder **`on_key_press()`** et **`on_key_release()`** pour modifier `change_x`/`change_y` avec les flèches.

**Rappels** :
- La vitesse est dans `self.player.attributes["Speed"]` (prévoyez une valeur par défaut si la clé manque).
- `arcade.key.LEFT`, `RIGHT`, `UP`, `DOWN` sont les codes touches.
- `on_key_press` → `change_x`/`change_y = ±Speed`. `on_key_release` → remettre à `0` la composante correspondante.

👉 **Complétez les `# TODO`**.


In [ ]:
# === Extraits à insérer/compléter dans main.py ===
from player import Player
from constants import *

class My_Game(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        self.map = None
        self.scene = None
        self.physics_engine = None
        self.camera = None
        self.player = None
        self.sprites_list = None
        self.gui = None

    def setup(self):
        arcade.set_background_color(arcade.csscolor.CORNFLOWER_BLUE)
        self.camera = arcade.Camera2D()
        self.camera.match_window()

        # Map & scène
        self.map = Map(self)
        self.map.setup()
        self.scene = arcade.Scene.from_tilemap(self.map.tile_map)

        # Liste des sprites
        self.sprites_list = arcade.SpriteList()

        # === TODO 1 : créer le joueur ===
        self.player = Player(PLAYER_FILE, PLAYER_SCALING, PLAYER_WIDTH, PLAYER_HEIGHT, PLAYER_SPRITE_COORDS)
        self.sprites_list.append(self.player)
        self.player.setup()

        # Moteur physique (collisions) pour le joueur
        self.map.set_collisions()

        # GUI
        self.gui = Gui(self)
        self.gui.setup()

    def on_key_press(self, key, modifiers):
        # === TODO 2 : gérer les flèches ===
        speed = self.player.attributes.get("Speed", 2)
        if key == arcade.key.LEFT:
            self.player.change_x = -speed
        ############ A COMPLETER (AUTRES DIRECTIONS) #############

    def on_key_release(self, key, modifiers):
        # === TODO 3 : remettre à 0 la composante relâchée ===
        if key == arcade.key.LEFT:
            self.player.change_x = 0
         ############ A COMPLETER (AUTRES DIRECTIONS) #############

    def on_update(self, delta_time):
        if self.physics_engine:
            self.physics_engine.update()
        if self.player:
            self.player.update()
        if self.camera and self.player:
            self.camera.position = (self.player.center_x, self.player.center_y)

    def on_draw(self):
        self.clear()
        if self.camera:
            self.camera.use()
        if self.scene:
            self.scene.draw()
        if self.sprites_list:
            self.sprites_list.draw()
        if self.gui:
            self.gui.draw()


## 4) Checklist de test <a id='check'></a>

- [ ] Le joueur **apparaît** à la bonne position (dépend du JSON et de `MAP_SCALING`).
- [ ] Les flèches **déplacent** le joueur (vérifiez `Speed`).
- [ ] Le joueur **ne sort pas** de la carte.
- [ ] Pas d’erreur de chemin (`PLAYER_FILE`, `PLAYER_CARACTERISTICS_FILE`).

🎯 **Objectif** : un joueur qui se **déplace**. Les **animations viendront après**.
